## Prepare Data for SDType and create input files.

In [ ]:
#imports
import random

In [ ]:
#get the type from the path files
def get_types(type_file_path, ds):
    if ds == "yago":
        #read yago transitive types
        from collections import defaultdict
        class_entity_dict = defaultdict(set)
        entity_class_dict = defaultdict(set)
        with open(type_file_path, "r") as yago_types:
            for line in yago_types:
                try:
                    x, entity, predicate, cl = line.split()
                    entity = entity.replace(">","").replace("<","")
                    cl = cl.replace(">","").replace("<","")
                    class_entity_dict[cl].add(entity)
                    entity_class_dict[entity].add(cl)
                except ValueError:
                    continue
    if ds == "freebase":
        from collections import defaultdict
        class_entity_dict = defaultdict(set)
        entity_class_dict = defaultdict(set)
        with open(type_file_path, "r") as fb_types:
            for line in fb_types:
                try:
                    entity, cl = line.split()
                    class_entity_dict[cl].add(entity)
                    entity_class_dict[entity].add(cl)
                except ValueError:
                    continue
        
    return class_entity_dict, entity_class_dict

In [ ]:
# Get Types for YAGO 3-10 Dataset from the original YAGO file.

entity_set = set()
with open (ntfiles[dataset],'r') as nt_file:
    for line in nt_file:
        s,p,o,_ = line.split()
        entity_set.add(s.replace("<http://www.yago-knowledge.org/","").replace(">",""))
        entity_set.add(o.replace("<http://www.yago-knowledge.org/","").replace(">",""))

with open('/data/yago/yago3-10TransitiveType.tsv', "w") as yago310_types:
    with open('/data/yago/yagoTransitiveType.tsv', "r") as yago_types:
            for line in yago_types:
                try:
                    x, entity, predicate, cl = line.split()
                    if entity.replace("<","").replace(">","") in entity_set:
                        yago310_types.write(line)
                except ValueError:
                    continue

In [ ]:
#insert file paths for type files, training datasets and output path for SDType training files.


output_path = '/SDType/'

ntfiles = {'yago' : '/data/yago/train.nt',
           'freebase' : '/data/freebase/train.nt'}
output_folder = {'yago' : '/data/yago/',
           'freebase' : '/data/freebase/'}

typefiles = {'yago' : '/data/yago/yago3-10TransitiveType.tsv',
           'freebase' : '/data/freebase/freebaseTypes.tsv'}

experiments = {'yago': {
           'level-1': ['wordnet_person_100007846', 'wordnet_organization_108008335', 'wordnet_body_of_water_109225146', 'wordnet_product_104007894'],
              'level-2-organization.txt': ['wordnet_musical_organization_108246613', 'wordnet_party_108256968', 'wordnet_enterprise_108056231', 'wordnet_nongovernmental_organization_108009834'],  
               'level-2-waterbody.txt': ['wordnet_stream_109448361', 'wordnet_lake_109328904', 'wordnet_ocean_109376198', 'wordnet_bay_109215664', 'wordnet_sea_109426788'],
               'level-2-person.txt': ['wordnet_artist_109812338', 'wordnet_officeholder_110371450', 'wordnet_writer_110794014', 'wordnet_scientist_110560637', 'wordnet_politician_110450303'], 
              'level-3-person-writer.txt': ['wordnet_journalist_110224578', 'wordnet_poet_110444194', 'wordnet_novelist_110363573', 'wordnet_scriptwriter_110564905', 'wordnet_dramatist_110030277', 'wordnet_essayist_110064405', 'wordnet_biographer_109855433'], 
              'level-3-person-scientist.txt': ['wordnet_social_scientist_110619642', 'wordnet_biologist_109855630', 'wordnet_physicist_110428004', 'wordnet_mathematician_110301261', 'wordnet_chemist_109913824', 'wordnet_linguist_110264437', 'wordnet_psychologist_110488865', 'wordnet_geologist_110127689', 'wordnet_computer_scientist_109951070', 'wordnet_research_worker_110523076'], 
             'level-3-person-player.txt': ['wordnet_football_player_110101634','wordnet_ballplayer_109835506','wordnet_soccer_player_110618342','wordnet_volleyball_player_110759047','wordnet_golfer_110136959'],
              'level-3-person-artist.txt': ['wordnet_painter_110391653', 'wordnet_sculptor_110566072', 'wordnet_photographer_110426749', 'wordnet_illustrator_109812068', 'wordnet_printmaker_110475687']
                    },
               'freebase':{
              'level-1': ['wordnet_person_100007846', 'wordnet_organization_108008335', 'wordnet_body_of_water_109225146', 'wordnet_product_104007894'],
             'level-2-organization.txt': ['wordnet_musical_organization_108246613', 'wordnet_party_108256968', 'wordnet_enterprise_108056231', 'wordnet_nongovernmental_organization_108009834'],  
               'level-2-person.txt': ['wordnet_artist_109812338', 'wordnet_officeholder_110371450', 'wordnet_writer_110794014', 'wordnet_scientist_110560637', 'wordnet_politician_110450303'], 
               'level-3-person-artist.txt': ['wordnet_painter_110391653', 'wordnet_sculptor_110566072', 'wordnet_photographer_110426749', 'wordnet_illustrator_109812068', 'wordnet_printmaker_110475687','wordnet_musician_110339966']           
                    }
               }


In [ ]:
for dataset in ntfiles.keys():
    #load classes
    try:
        class_entity_dict, entity_class_dict = get_types(typefiles[dataset], dataset)
    except Exception as e:
        print(str(e))
        break
    #get entity list from file
    entity_set = set()
    with open (ntfiles[dataset],'r') as nt_file:
        for line in nt_file:
            s,p,o,_ = line.split()
            if dataset == 'freebase':
                entity_set.add(s.replace("<http://www.freebase.com","").replace(">",""))
                entity_set.add(o.replace("<http://www.freebase.com","").replace(">",""))
            else:
                entity_set.add(s.replace("<http://www.yago-knowledge.org/","").replace(">",""))
                entity_set.add(o.replace("<http://www.yago-knowledge.org/","").replace(">",""))
            
    entity_list = list(entity_set)
    print("Loaded dataset: {}".format(dataset))
    
    
    for e in experiments[dataset].keys():
        input_classes = experiments[dataset][e]
        relevant_types = []
        relevant_entities = set()
        # label array annotating each entity with classes
        labels = []
        for c in input_classes:
            print("Check for class {}".format(c))
            class_size_counter = 0
            for i, entity in enumerate(entity_list):
                #if entity in class_entity_dict[c] and entity in new_entity_list:
                if entity in class_entity_dict[c]:
                    type_nt_line = "{}\t{}\n".format(entity,c)
                    relevant_types.append(type_nt_line)
                    relevant_entities.add(entity)

        #split into train and testset
        random.shuffle(relevant_types)
        train_length = int(len(relevant_types)*0.8)
        train_data = relevant_types[:train_length]
        test_data = relevant_types[train_length:]
        with open (output_folder[dataset]+e+'train.nt','w') as train_file:
            for line in train_data:
                train_file.write(line)
        with open (output_folder[dataset]+e+'test.nt','w') as test_file:
            for line in test_data:
                test_file.write(line)
        with open (output_folder[dataset]+e+'training_triples.txt','w') as training_triples:
            with open (ntfiles[dataset],'r') as nt_file:
                for line in nt_file:
                    s,p,o,_ = line.split()
                    if dataset == 'freebase':
                        if s.replace("<http://www.freebase.com","").replace(">","") in relevant_entities or s.replace("<http://www.freebase.com","").replace(">","") in relevant_entities:
                            training_triples.write("{}\t{}\t{}\n".format(s.replace("<http://www.freebase.com","").replace(">",""), p, s.replace("<http://www.freebase.com","").replace(">","")))
                    else:
                        if s.replace("<http://www.yago-knowledge.org/","").replace(">","") in relevant_entities or o.replace("<http://www.yago-knowledge.org/","").replace(">","") in relevant_entities:
                            training_triples.write("{}\t{}\t{}\n".format(s.replace("<http://www.yago-knowledge.org/","").replace(">",""), p, o.replace("<http://www.yago-knowledge.org/","").replace(">","")))

## Evaluate the Output Files

The input to this code snipped is the output of SDType: https://github.com/HeikoPaulheim/sd-type-validate


In [ ]:
infiles = ["level-1train.nt_sdtype.csv","level-2-organization.txttrain.nt_sdtype.csv","level-2-waterbody.txttrain.nt_sdtype.csv","level-2-person.txttrain.nt_sdtype.csv","level-3-person-writer.txttrain.nt_sdtype.csv","level-3-person-scientist.txttrain.nt_sdtype.csv","level-3-person-player.txttrain.nt_sdtype.csv","level-3-person-artist.txttrain.nt_sdtype.csv"]
testfiles = ["level-1test.nt","level-2-organization.txttest.nt","level-2-waterbody.txttest.nt","level-2-person.txttest.nt","level-3-person-writer.txttest.nt","level-3-person-scientist.txttest.nt","level-3-person-player.txttest.nt","level-3-person-artist.txttest.nt"]
import pandas as pd
import numpy as np
df = pd.DataFrame(columns=['Experiment','Treshold','Precision', 'Recall'])
from collections import defaultdict
#infiles = ["level-3-person-artist.txttrain.nt_sdtype.csv"]
#testfiles = ["level-3-person-artist.txttest.nt"]

print("loading....")
#one loop per experiment
for testfile, infile in zip(testfiles,infiles):
    gold_data = set()
    no_of_correct_results_perclass = defaultdict(lambda: 0)
    testset_entities = set()
    try:
        with open ('/fb15k-237/'+testfile,'r') as test_file:
            for line in test_file:
                resource, t = line.split("\t")
                testset_entities.add(resource)
                t = t.replace("\n","")
                no_of_correct_results_perclass[t] += 1
                gold_data.add(resource+t)
    except FileNotFoundError:
        print("Error")
        continue
    print("Read test data")
    #read test dataset
    
    #one experiment per threshold
    predictions = defaultdict(dict)
    with open ('/fb15k-237/'+infile,'r') as prediction_file:
        for line in prediction_file:
            entity,e_class, confidence  = line.split(" ")
            entity = entity.replace(">","").replace("<","")
            e_class = e_class.replace(">","").replace("<","")
                
            predictions[e_class][entity] = float(confidence)

    print("Read prediction data")
    top_f1 = 0.0
    for threshold in np.arange(0.0, 1, 0.001):
        no_prediction_over_threshold = 0
        avg_f1 = 0.0
        avg_p = 0.0
        avg_r = 0.0
        #compute weight averaged f1 measure
        for e_class in predictions.keys():
            correct_in_sdtypes = 0
            all_predictions = 0 
            for entity, confidence in predictions[e_class].items():    
                prediction = entity+e_class
                if prediction in gold_data and confidence>= threshold:
                    correct_in_sdtypes += 1
                if confidence>= threshold and entity in testset_entities:
                    all_predictions += 1
            try:
                p = correct_in_sdtypes/all_predictions
                r = correct_in_sdtypes/no_of_correct_results_perclass[e_class]
                f1 = (2*p*r)/(p+r)
            except ZeroDivisionError:
                #print("Error")
                p = 0
                r = 0
                f1= 0
            #if f1 > 0.0 and p> 0 and r > 0:
            no_prediction_over_threshold += no_of_correct_results_perclass[e_class]
            avg_p += p*no_of_correct_results_perclass[e_class]
            avg_r += r*no_of_correct_results_perclass[e_class]
                
            avg_f1 += f1*no_of_correct_results_perclass[e_class]
            if f1 > 1.0:
                raise ValueError(f"F1 should not be {f1}")
        if no_prediction_over_threshold >0:
            avg_f1 = avg_f1/no_prediction_over_threshold
            avg_p = avg_p/no_prediction_over_threshold
            avg_r = avg_r/no_prediction_over_threshold
            new_row = pd.Series({'Experiment':testfile.replace("test.nt",""),'Treshold': threshold, 'overTH':no_prediction_over_threshold, 'Precision': avg_p, 'Recall': avg_r})
            df = df.append(new_row, ignore_index=True)
        
        #only keep if the current f1 measure is better than the existing one
        if avg_f1>= top_f1:
            top_f1 = avg_f1




    
    

In [ ]:

df_new = df[['Experiment','Precision', 'Recall']].drop_duplicates()
df_new = df_new.replace({'level-1': 'Level-1'})
df_new = df_new.replace({'level-2-organization.txt': 'Level-2-Organizations'})
df_new = df_new.replace({'level-2-waterbody.txt': 'Level-2-Waterbodies'})
df_new = df_new.replace({'level-2-person.txt': 'Level-2-Persons'})
df_new = df_new.replace({'level-3-person-writer.txt': 'Level-3-Writers'})
df_new = df_new.replace({'level-3-person-scientist.txt': 'Level-3-Scientists'})
df_new = df_new.replace({'level-3-person-player.txt': 'Level-3-Players'})
df_new = df_new.replace({'level-3-person-artist.txt': 'Level-3-Artists'})



import seaborn as sns
sns.set_style("whitegrid")

ax = sns.lineplot(data=df_new, x="Recall", y="Precision", hue="Experiment", ci=None, markers=True)
